# Analyzing Marketplace-collected commute data
As part of the test for Marketplace's episode on "Canada's Worst Commute", we asked our audience to submit their commute to us. We then tracked this commute for a minimum of 30 days. This notebook analyzes those results.

In [15]:
import gcsfs
import pandas as pd

Now we read in our data from the Google bucket.

In [16]:
# Set your project ID and bucket name
project_id = 'dig-es-nws-gemini-projects'
bucket_name = 'marketplace-commutes'

# Initialize gcsfs
gcs = gcsfs.GCSFileSystem(project=project_id)

# List all CSV files in the bucket
files = gcs.glob(f'gs://{bucket_name}/*.csv')

print(files)

# Read all CSV files into a list of DataFrames
all_dfs = []
for f in files:
    df_temp = pd.read_csv(f"gs://{f}")
    # Extract timezone from filename
    filename = f.split('/')[-1]
    if 'atlantic' in filename:
        df_temp['timezone'] = 'America/Halifax'
    elif 'central' in filename:
        df_temp['timezone'] = 'America/Winnipeg'
    elif 'eastern' in filename:
        df_temp['timezone'] = 'America/Toronto'
    elif 'mountain' in filename:
        df_temp['timezone'] = 'America/Edmonton'
    elif 'pacific' in filename:
        df_temp['timezone'] = 'America/Vancouver'
    else:
        df_temp['timezone'] = 'UTC' # Default or handle as needed
    all_dfs.append(df_temp)


# Concatenate all DataFrames into a single DataFrame
df = pd.concat(all_dfs, ignore_index=True)

# Create a unique identifier for each route
df['route_id'] = df['origin'] + ' to ' + df['destination']

# --- Create timestamp_local column converting 'timestamp' into local time per-row ---
# Parse 'timestamp' into UTC-aware Timestamps (assume UTC if timezone-naive)
df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True, errors='coerce')

# Helper to convert a UTC-aware timestamp to a per-row timezone with safe fallback
def _to_local(ts, tz):
    try:
        if pd.isna(ts):
            return pd.NaT
        if pd.isna(tz) or tz == '':
            tz = 'UTC'
        return ts.tz_convert(tz)
    except Exception:
        # If conversion fails (invalid tz string), return the original UTC timestamp
        try:
            return ts.tz_convert('UTC')
        except Exception:
            return pd.NaT

# Apply conversion per-row (vectorized conversion per value isn't possible when tz varies by row)
if 'timezone' in df.columns:
    df['timestamp_local'] = df.apply(lambda r: _to_local(r['timestamp'], r['timezone']), axis=1)
else:
    # If timezone column is missing, fallback to UTC
    df['timestamp_local'] = df['timestamp']

# Display a quick sample to check results
display(df.head())

['marketplace-commutes/commute_routes_atlantic.csv', 'marketplace-commutes/commute_routes_central.csv', 'marketplace-commutes/commute_routes_eastern.csv', 'marketplace-commutes/commute_routes_mountain.csv', 'marketplace-commutes/commute_routes_pacific.csv']


,origin,destination,travel_mode,distance_km,duration_min,warnings,line_geometry,timestamp,timezone,route_id,timestamp_local
0,"Larry Uteck, Halifax",Hospital in Halifax,DRIVE,10.113,20,NaN,"LINESTRING (-63.67528 44.70207, -63.67451 44.7...",2025-08-26 20:10:05+00:00,America/Halifax,"Larry Uteck, Halifax to Hospital in Halifax",2025-08-26 17:10:05-03:00
1,"Windsor Street, Halifax","St. Margaret's Bay Road, Halifax",DRIVE,5.678,13,NaN,"LINESTRING (-63.60553 44.65555, -63.60527 44.6...",2025-08-26 20:10:05+00:00,America/Halifax,"Windsor Street, Halifax to St. Margaret's Bay ...",2025-08-26 17:10:05-03:00
2,"Armdale, Halifax","Bayers Lake, Halifax",DRIVE,7.775,12,NaN,"LINESTRING (-63.60788 44.63002, -63.60795 44.6...",2025-08-26 20:10:06+00:00,America/Halifax,"Armdale, Halifax to Bayers Lake, Halifax",2025-08-26 17:10:06-03:00
3,"North End, Halifax","Macdonald Bridge, Halifax",BICYCLE,1.723,10,NaN,"LINESTRING (-63.60227 44.65962, -63.60234 44.6...",2025-08-26 20:10:06+00:00,America/Halifax,"North End, Halifax to Macdonald Bridge, Halifax",2025-08-26 17:10:06-03:00
4,"Larry Uteck, Halifax",Hospital in Halifax,DRIVE,10.113,21,NaN,"LINESTRING (-63.67528 44.70207, -63.67451 44.7...",2025-08-26 20:20:05+00:00,America/Halifax,"Larry Uteck, Halifax to Hospital in Halifax",2025-08-26 17:20:05-03:00


I want to create a route_id from the origin and destination points in our routes file here so I can stitch it onto the data I'm reading in that we've collected.

How many routes are we tracking?

In [17]:
len(df["route_id"].unique())

55

Now I'd like to stitch on some extra data so we can tell which route was submitted by which person. This will add a "notes" column.

In [18]:
notes_df = pd.read_csv("data/dexter/commute_routes.csv")

notes_df['route_id'] = notes_df['origin'] + ' to ' + notes_df['destination']

In [19]:
# Merge notes from the local CSV
df = pd.merge(df, notes_df[['route_id', 'notes']], on='route_id', how='left')

# Display the first few rows of the combined DataFrame
df.head(2)

,origin,destination,travel_mode,distance_km,duration_min,warnings,line_geometry,timestamp,timezone,route_id,timestamp_local,notes
0,"Larry Uteck, Halifax",Hospital in Halifax,DRIVE,10.113,20,NaN,"LINESTRING (-63.67528 44.70207, -63.67451 44.7...",2025-08-26 20:10:05+00:00,America/Halifax,"Larry Uteck, Halifax to Hospital in Halifax",2025-08-26 17:10:05-03:00,NaN
1,"Windsor Street, Halifax","St. Margaret's Bay Road, Halifax",DRIVE,5.678,13,NaN,"LINESTRING (-63.60553 44.65555, -63.60527 44.6...",2025-08-26 20:10:05+00:00,America/Halifax,"Windsor Street, Halifax to St. Margaret's Bay ...",2025-08-26 17:10:05-03:00,NaN


Now let's get the hour from the localized timestamp and put it in a new column.

In [20]:
df['hour'] = df['timestamp_local'].apply(lambda ts: ts.hour if pd.notnull(ts) else pd.NA)

Now for some cleaning. I had changed the origin or destination for a few routes mid-collection, which means we need to change them back here. Otherwise, they'll show up as two different routes.

In [21]:
df["route_id"] = df["route_id"].str.replace("44.41906079312099, -80.09223079949398", "Stayner, Ontario L0M 1S0")
df["route_id"] = df["route_id"].str.replace("44.672247457480196, -63.478352018683125", "Cole Harbour, Nova Scotia")

# Drop this route because it only collected a few times, and was returning wonky results.
# I replaced it with a route using lat/lon coordinates instead.
df = df[df['route_id'] != 'Westshore, Victoria to Downtown Victoria']

Let's take a look at a few as an example.

In [22]:
df["route_id"].head(3).to_list()

['Larry Uteck, Halifax to Hospital in Halifax',
 "Windsor Street, Halifax to St. Margaret's Bay Road, Halifax",
 'Armdale, Halifax to Bayers Lake, Halifax']

Now let's just see when this data starts and ends. Note I'm using the UTC timestamp to do this, and the "real" time I started and ended would be Toronto time.

In [23]:
(display(pd.to_datetime(df["timestamp"], utc=True, errors='coerce').min(),
         pd.to_datetime(df["timestamp"], utc=True, errors='coerce').max()
         )
 )

Timestamp('2025-08-26 20:10:05+0000', tz='UTC')

Timestamp('2025-10-20 14:10:14+0000', tz='UTC')

In [24]:
# Filter for off-peak hours
off_peak_df = df[~((df['hour'] >= 7) & (df['hour'] < 9) | (df['hour'] >= 17) & (df['hour'] < 19))]

# Calculate the mean travel time for each route during off-peak hours
off_peak_stats = off_peak_df.groupby(['route_id', 'travel_mode'])['duration_min'].agg(['mean']).reset_index()
off_peak_stats.columns = ['route_id', 'travel_mode', 'mean_off_peak_duration_min']

# Filter for peak hours
peak_df = df[((df['hour'] >= 8) & (df['hour'] < 10) | (df['hour'] >= 17) & (df['hour'] < 19))]

# Calculate the mean and 90th percentile travel time for each route during peak hours
peak_stats_mean = peak_df.groupby(['route_id', 'travel_mode'])['duration_min'].mean().reset_index().rename(columns={'duration_min': 'mean_peak_duration_min'})
peak_stats_p90 = peak_df.groupby(['route_id', 'travel_mode'])['duration_min'].quantile(0.9).reset_index().rename(columns={'duration_min': 'p90_peak_duration_min'})

# Merge the stats: off-peak mean, peak mean, and peak p90
final_stats = off_peak_stats.merge(peak_stats_mean, on=['route_id', 'travel_mode'], how='left').merge(peak_stats_p90, on=['route_id', 'travel_mode'], how='left')

# Add calculated columns: mean difference and p90 difference
final_stats['peak_vs_off_peak_mean_diff'] = final_stats['mean_peak_duration_min'] - final_stats['mean_off_peak_duration_min']
final_stats['p90_vs_off_peak_diff'] = final_stats['p90_peak_duration_min'] - final_stats['mean_off_peak_duration_min']

# Calculate the score and sort by it
final_stats = final_stats.sort_values(by='peak_vs_off_peak_mean_diff', ascending=False)

# Merge notes from the read-in commute_routes.csv (if available)
if 'notes' in locals() or 'notes_df' in globals():
    try:
        final_stats = final_stats.merge(notes_df[['route_id', 'notes']], on='route_id', how='left')
    except Exception:
        # if merge fails for any reason, create an empty notes column
        final_stats['notes'] = ''
else:
    final_stats['notes'] = ''

# Display the final table with notes
display(final_stats.head(20))

,route_id,travel_mode,mean_off_peak_duration_min,mean_peak_duration_min,p90_peak_duration_min,peak_vs_off_peak_mean_diff,p90_vs_off_peak_diff,notes
0,"Highway 401, Mississauga to Highway 401 and Do...",DRIVE,39.693694,47.942610,62.0,8.248916,22.306306,NaN
1,"Cloverdale, Surrey to Richmond/Vancouver",DRIVE,53.036126,60.385321,79.0,7.349195,25.963874,NaN
2,"Port Coquitlam, BC to Richmond Sea Island, BC",DRIVE,50.405885,55.141956,70.0,4.736071,19.594115,NaN
3,"Gardiner Expressway and Highway 427, Toronto t...",DRIVE,18.627592,21.996970,27.0,3.369377,8.372408,NaN
4,"The Beaches, Toronto, ON to CBC Toronto, Canad...",DRIVE,22.388739,25.500000,32.0,3.111261,9.611261,Submitted by Dana Dragone
5,"49.1628474739296, -122.84518311047466 to Coqui...",DRIVE,29.893353,32.797743,40.0,2.904390,10.106647,88th avenue in Surrey
6,"Stayner, Ontario L0M 1S0 to 43.67371164802397,...",DRIVE,101.396035,104.024371,114.0,2.628336,12.603965,NaN
7,"Larry Uteck, Halifax to Hospital in Halifax",DRIVE,16.672339,19.191667,26.0,2.519328,9.327661,NaN
8,"Riverside South, Ottawa to Downtown Ottawa",DRIVE,26.449902,28.962121,35.0,2.512219,8.550098,NaN
9,"Bathurst and St. Clair, Toronto to 43.73014981...",DRIVE,17.933108,20.412736,27.0,2.479628,9.066892,Submitted by Steven Dennis


In [25]:
final_stats.to_csv("results-dexter.csv")

In [26]:
final_stats["travel_mode"].value_counts()

travel_mode
DRIVE      32
TRANSIT    14
BICYCLE     9
Name: count, dtype: int64